## 基础理论部分

#### 0. Can you come up out 3 sceneraies which use AI methods? 

Ans: 火车站的人脸识别；手机上的语音助手；特斯拉的自动驾驶

#### 1. How do we use Github; Why do we use Jupyter and Pycharm;

Ans: GitHub传道授业解惑，提交作业；Jupyter做学习笔记，方便解析整个过程，而Pycharm是IDE，用于开发项目

#### 2. What's the Probability Model?

Ans: 用来计算输入出现在总体样本的概率

#### 3. Can you came up with some sceneraies at which we could use Probability Model?

Ans: 输入法的推荐后面的字

#### 4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match?

Ans: 用来判断事物的可能性或者预测事情的走向；样本需要足够大并保证一定的正确性

#### 5. What's the Language Model;

Ans: 按照给定的pattern生成句子，判断哪个句子更为真实

#### 6. Can you came up with some sceneraies at which we could use Language Model?


Ans: 语音识别；机器翻译

#### 7. What's the 1-gram language model;

Ans: 单个词出现的概率

#### 8. What's the disadvantages and advantages of 1-gram language model;

Ans: Dis——无法关联性预测；Adv——可提供大量基础数据

#### 9. What't the 2-gram models;

Ans: 用来根据前1个item来预测后一个item

## 编程实践部分

#### 1. 设计你自己的句子生成器

In [169]:
import random
import re
import jieba
from collections import Counter

In [170]:
sentence = '''
sentence = people verb noun
people = Alice | Beyond | Trump
verb = do | doing
do = plays | sings
doing = is act
act = playing | singing
noun = basketball | song | music
'''

In [171]:
ans = '''
ans = a verb b noun
a = you | he | she | I
verb = turn | ask | find
b = dracula | fairy
noun = 1 inter
1 = for | to
inter = help | advice | suggestion
'''

In [172]:
def grammar(gram_str, split='='):
    grammar = {}
    for line in gram_str.split('\n'):
        if not line.strip(): continue
        l, r = line.split(split)
        grammar[l.strip()] = [s.split() for s in r.split('|')]
    return grammar

In [215]:
def generate(gram, target):
    if target not in gram: return target
    expanded = [generate(gram, t) for t in random.choice(gram[target])]
    #return ' '.join(x for x in expanded)
    return ''.join(x for x in expanded)

In [174]:
generate(grammar(sentence), 'sentence')

'Trump plays song'

In [175]:
generate(grammar(ans), 'ans')

'he turn fairy for suggestion'

In [206]:
def generate_n(gram, target, n):
    result = []
    for i in range(n + 1):
        inter = generate(grammar(gram), target)
        result.append(inter)
    return result

In [208]:
a = generate_n(ans, 'ans', 20)
a

['she find dracula for advice',
 'he find fairy to suggestion',
 'I find fairy to advice',
 'I turn fairy to suggestion',
 'you find fairy to suggestion',
 'I ask dracula to advice',
 'you find dracula for suggestion',
 'she find fairy to advice',
 'he ask fairy to advice',
 'she turn dracula to suggestion',
 'she find dracula to suggestion',
 'she turn fairy for advice',
 'he find dracula to advice',
 'I ask fairy to suggestion',
 'you turn fairy for help',
 'you turn fairy to advice',
 'I find dracula to suggestion',
 'she turn fairy to suggestion',
 'I ask fairy for advice',
 'he ask dracula to help',
 'she find dracula for advice']

#### 2. 使用新数据源完成语言模型的训练

In [177]:
filename = 'D:\\GitHub\\NLP\\data\\train.txt'

In [178]:
with open(filename, encoding='utf-8') as f:
    content = f.read()

In [179]:
len(content)

1343520

In [180]:
def token(string):
    return re.findall('\w+', string)

In [181]:
cut = Counter(jieba.cut(content))

In [182]:
cut.most_common()[:9]

[(' ', 241272),
 ('++', 77334),
 ('$', 38679),
 ('-', 13317),
 ('\n', 12889),
 ('?', 12876),
 ('？', 12736),
 ('insurance', 11842),
 ('Insurance', 10011)]

In [183]:
content_clean = [''.join(token(content))]

In [184]:
with open('train_clean.txt', 'w') as f:
    for a in content_clean:
        f.write(a + '\n')

In [185]:
def cut(string):
    return list(jieba.cut(string))

In [186]:
TOKEN = []

In [187]:
for i, line in enumerate(open('train_clean.txt')):
    TOKEN += cut(line)

In [188]:
TOKEN = [str(t) for t in TOKEN]

In [189]:
TOKEN_2_GRAM = [''.join(TOKEN[i:i + 2]) for i in range(len(TOKEN[:-2]))]

In [190]:
len(TOKEN_2_GRAM)

87966

In [191]:
words_count = Counter(TOKEN_2_GRAM)

In [193]:
def prob_2(word1, word2):
    if word1 + word2 in words_count: return words_count[word1 + word2] / len(TOKEN_2_GRAM)
    else:
        return 1 / len(TOKEN_2_GRAM)

In [194]:
prob_2('买','保险')

2.273605711297547e-05

In [195]:
prob_2('卖','糖')

1.1368028556487734e-05

In [196]:
prob_2('去','游玩')

1.1368028556487734e-05

In [197]:
prob_2('什么','是')

0.013061864811404407

#### 3. 获得最优质的的语言

当我们能够生成随机的语言并且能判断之后，我们就可以生成更加合理的语言了。请定义 generate_best 函数，该函数输入一个语法 + 语言模型，能够生成**n**个句子，并能选择一个最合理的句子: 



In [198]:
sorted([1, 3, 5, 2])

[1, 2, 3, 5]

这个函数接受一个参数key，这个参数接受一个函数作为输入，例如

In [199]:
sorted([(2, 5), (1, 4), (5, 0), (4, 4)], key=lambda x: x[0])

[(1, 4), (2, 5), (4, 4), (5, 0)]

能够让list按照第0个元素进行排序.

In [200]:
sorted([(2, 5), (1, 4), (5, 0), (4, 4)], key=lambda x: x[1])

[(5, 0), (1, 4), (4, 4), (2, 5)]

能够让list按照第1个元素进行排序.

In [201]:
sorted([(2, 5), (1, 4), (5, 0), (4, 4)], key=lambda x: x[1], reverse=True)

[(2, 5), (1, 4), (4, 4), (5, 0)]

能够让list按照第1个元素进行排序, 但是是递减的顺序。

In [209]:
def get_probablity(sentence):
    words = cut(sentence)   
    sentence_pro = 1    
    for i, word in enumerate(words[:-1]):
        next_ = words[i + 1]
        probability = prob_2(word, next_)        
        sentence_pro *= probability    
    return sentence_pro

In [220]:
def generate_best(): # you code here
    sentence = '''
    sentence = a verb noun
    a = 医疗保险 | 债权人 | 费用
    verb = 有 | 是 | 涵盖 
    noun = 什么 | 时间 | 特点 | 多少
    '''
    result = generate_n(sentence, 'sentence', 6)
    pro_list = []
    for i in result:
        pro = get_probablity(i)
        pro_list.append((pro, i))
    pro_list = sorted(pro_list, key=lambda x: x[0], reverse=True)
    return pro_list

In [221]:
generate_best()

[(8.238544670396437e-07, '医疗保险是什么'),
 (2.326177318700171e-09, '医疗保险涵盖特点'),
 (3.876962197833618e-10, '费用涵盖多少'),
 (1.292320732611206e-10, '费用涵盖时间'),
 (1.292320732611206e-10, '费用涵盖特点'),
 (1.292320732611206e-10, '债权人有时间'),
 (1.292320732611206e-10, '债权人涵盖时间')]

Q: 这个模型有什么问题？ 你准备如何提升？ 

Ans: 混有中英文，需要将英文和中文处理下